# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-15-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-15-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-16 05:22:40,33.93911,67.709953,53831,2324,45434,6073.0,Afghanistan,138.282286,4.317215
1,NaN,NaN,NaN,Albania,2021-01-16 05:22:40,41.15330,20.168300,66635,1265,39625,25745.0,Albania,2315.484050,1.898402
2,NaN,NaN,NaN,Algeria,2021-01-16 05:22:40,28.03390,1.659600,103381,2827,70188,30366.0,Algeria,235.754940,2.734545
3,NaN,NaN,NaN,Andorra,2021-01-16 05:22:40,42.50630,1.521800,8946,88,8116,742.0,Andorra,11578.334304,0.983680
4,NaN,NaN,NaN,Angola,2021-01-16 05:22:40,-11.20270,17.873900,18679,428,16120,2131.0,Angola,56.833347,2.291343


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,53105,53207,53332,53400,53489,53538,53584,53584,53775,53831
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,61008,61705,62378,63033,63595,63971,64627,65334,65994,66635
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,101120,101382,101657,101913,102144,102369,102641,102860,103127,103381


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2244,2253,2257,2264,2277,2288,2301,2301,2314,2324
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1217,1223,1230,1233,1241,1247,1252,1256,1261,1265
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2786,2792,2798,2803,2807,2812,2816,2819,2822,2827


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,42666,43291,43440,43740,43948,44137,44608,44850,45298,45434
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,36102,36535,36971,37327,37648,37981,38421,38860,39246,39625
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,68383,68589,68802,69011,69212,69403,69608,69791,69992,70188


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4645,4705,4770,4847,4879,4902,4970,4998,5075,5103
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,14656,14845,15052,15202,15327,15417,15572,15701,15841,16002
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1597,1614,1634,1648,1658,1663,1679,1685,1696,1712


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,50,50,50,53,54,55,55,55,55,55
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,171,171,171,173,173,173,175,175,177,179
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,35,35,35,35,35,35,35,35,36,36


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
650,1001.0,Autauga,Alabama,US,2021-01-16 05:22:40,32.539527,-86.644082,5103,55,0,5048.0,"Autauga, Alabama, US",9133.866724,1.077797
687,1075.0,Lamar,Alabama,US,2021-01-16 05:22:40,33.779950,-88.096680,1166,19,0,1147.0,"Lamar, Alabama, US",8446.215139,1.629503
688,1077.0,Lauderdale,Alabama,US,2021-01-16 05:22:40,34.901719,-87.656247,7451,83,0,7368.0,"Lauderdale, Alabama, US",8035.242481,1.113944


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,23524081,391955,0
India,10542841,152093,10179715
Brazil,8393492,208246,7438075


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,23524081,391955,0,23132126,2021-01-16 05:22:40,37.936303,-91.379001
India,10542841,152093,10179715,211033,2021-01-16 05:22:40,23.088275,81.806127
Brazil,8393492,208246,7438075,747171,2021-01-16 05:22:40,-12.669522,-48.480493
Russia,3483531,63558,2879082,540891,2021-01-16 05:22:40,54.546312,62.120860
United Kingdom,3325642,87448,7691,3230503,2021-01-16 05:22:40,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,2916309,32851,0
Texas,2069286,31693,0
Florida,1548067,23799,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,2916309,32851,0,2883458,2021-01-16 05:22:40,37.843962,-120.728594
Texas,2069286,31693,0,2037593,2021-01-16 05:22:40,31.660643,-98.653069
Florida,1548067,23799,0,1524268,2021-01-16 05:22:40,28.940755,-82.700744
New York,1212922,40637,0,1172285,2021-01-16 05:22:40,42.544151,-75.474183
Illinois,1059324,19873,0,1039451,2021-01-16 05:22:40,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,989928,13489,0
Illinois,Cook,429270,8939,0
Arizona,Maricopa,407631,6320,0
Florida,Miami-Dade,339434,4528,0
Texas,Harris,274899,3691,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,989928,13489,0,976439,2021-01-16 05:22:40,34.308284,-118.228241,6037.0
Illinois,Cook,429270,8939,0,420331,2021-01-16 05:22:40,41.841448,-87.816588,17031.0
Arizona,Maricopa,407631,6320,0,401311,2021-01-16 05:22:40,33.348359,-112.491815,4013.0
Florida,Miami-Dade,339434,4528,0,334906,2021-01-16 05:22:40,25.611236,-80.551706,12086.0
Texas,Harris,274899,3691,0,271208,2021-01-16 05:22:40,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-13,53584,65334,102860,8818,18425,176,1757429,163128,28660,385750,...,3220953,28475,77716,1,117811,1521,149769,2109,31100,24256
2021-01-14,53775,65994,103127,8868,18613,184,1770715,163576,28669,389260,...,3269757,29989,77777,1,118415,1531,150505,2110,32800,25368
2021-01-15,53831,66635,103381,8946,18679,184,1783047,163972,28689,390788,...,3325642,30946,77904,1,118856,1536,151142,2111,34278,26109


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-13,2301,1256,2819,87,424,5,44983,2951,909,6868,...,84910,275,618,0,1084,35,1658,612,509,589
2021-01-14,2314,1261,2822,88,425,6,45125,2963,909,6921,...,86163,280,618,0,1090,35,1665,612,514,636
2021-01-15,2324,1265,2827,88,428,6,45227,2974,909,6987,...,87448,291,619,0,1095,35,1687,612,527,666


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-13,44850,38860,69791,8070,15631,152,1536423,151246,25875,359692,...,7430,20733,76197,1,111616,1369,134977,1419,21568,13658
2021-01-14,45298,39246,69992,8091,16008,156,1549490,151849,25884,362931,...,7580,21685,76276,1,111999,1369,136247,1419,22504,14714
2021-01-15,45434,39625,70188,8116,16120,156,1561963,152320,25897,365214,...,7691,22423,76388,1,112320,1380,137648,1419,24105,15414


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4645,4705,4770,4847,4879,4902,4970,4998,5075,5103
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1098,1114,1124,1132,1137,1146,1153,1157,1168,1166
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,6839,6930,7051,7149,7191,7214,7296,7350,7402,7451
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2181,2217,2263,2315,2324,2340,2351,2368,2393,2404
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,10964,11179,11355,11498,11565,11626,11812,11983,12120,12261


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-13,410995,50816,0,641729,262020,2843062,366774,220576,67173,49,...,665499,2040751,314817,9368,2166,412545,281202,104392,558020,48289
2021-01-14,414583,51157,0,649040,264511,2883699,369177,221544,68065,49,...,670482,2068637,317559,9573,2166,417839,283777,105219,561160,48701
2021-01-15,417528,51466,0,658186,267635,2916309,371458,223422,68727,49,...,676039,2069286,320102,9734,2166,422634,285970,106649,563866,48909


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-01-13        4998   15701    1685  2109   5170     966   1651   10745   
2021-01-14        5075   15841    1696  2113   5219     971   1669   10863   
2021-01-15        5103   16002    1712  2130   5264     981   1679   10982   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-13         2734     1569  ...    2346    500     2721      631   
2021-01-14         2757     1583  ...    2368    509     2740      632   
2021-01-15         2778     1596  ...    2396    515     2746      633   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-13           3222  2502  1774          0      830    601  
2021-01-14           3250  2579  1794          0      837    602  
2021-01-15           3262  2605  1806          0      840    602  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-13,5760,226,0,10673,4186,31684,5285,6536,994,0,...,8148,31277,1449,158,24,5552,3838,1671,5679,522
2021-01-14,5945,228,0,10855,4228,32318,5316,6553,995,0,...,8232,31693,1460,162,24,5626,3876,1702,5728,522
2021-01-15,6030,229,0,11040,4263,32851,5343,6594,1002,0,...,8311,31693,1472,163,24,5656,3903,1733,5770,522


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-01-13          55     175      35   48     80      28     57     197   
2021-01-14          55     177      36   47     80      28     58     197   
2021-01-15          55     179      36   47     83      28     58     200   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-13           66       29  ...      16      9       17        7   
2021-01-14           69       30  ...      16      9       17        7   
2021-01-15           69       30  ...      16      9       17        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-13             26     4     9          0       23      4  
2021-01-14             26     4     9          0       23      4  
2021-01-15             26     4     9          0       23      4  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-13,0.000000,0.010940,0.002134,0.015665,0.004470,0.000000,0.007294,0.002982,0.000349,0.004994,...,0.015020,0.022589,0.000682,0.0,0.004365,0.000658,0.005377,0.000949,0.045132,0.043763
2021-01-14,0.003564,0.010102,0.002596,0.005670,0.010204,0.045455,0.007560,0.002746,0.000314,0.009099,...,0.015152,0.053169,0.000785,0.0,0.005127,0.006575,0.004914,0.000474,0.054662,0.045844
2021-01-15,0.001041,0.009713,0.002463,0.008796,0.003546,0.000000,0.006964,0.002421,0.000698,0.003925,...,0.017091,0.031912,0.001633,0.0,0.003724,0.003266,0.004232,0.000474,0.045061,0.029210


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-13,0.000000,0.003195,0.001065,0.011628,0.004739,0.0,0.003010,0.003400,0.0,0.007186,...,0.018814,0.022305,0.001621,NaN,0.005566,0.0,0.017178,0.0,0.028283,0.068966
2021-01-14,0.005650,0.003981,0.001064,0.011494,0.002358,0.2,0.003157,0.004066,0.0,0.007717,...,0.014757,0.018182,0.000000,NaN,0.005535,0.0,0.004222,0.0,0.009823,0.079796
2021-01-15,0.004322,0.003172,0.001772,0.000000,0.007059,0.0,0.002260,0.003712,0.0,0.009536,...,0.014914,0.039286,0.001618,NaN,0.004587,0.0,0.013213,0.0,0.025292,0.047170


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-13,0.005425,0.011426,0.002629,0.017654,0.007671,0.000000,0.005604,0.004276,0.000387,0.005957,...,0.011159,0.031544,0.000827,0.0,0.003967,0.005878,0.010314,0.002119,0.023441,0.019558
2021-01-14,0.009989,0.009933,0.002880,0.002602,0.024119,0.026316,0.008505,0.003987,0.000348,0.009005,...,0.020188,0.045917,0.001037,0.0,0.003431,0.000000,0.009409,0.000000,0.043398,0.077317
2021-01-15,0.003002,0.009657,0.002800,0.003090,0.006997,0.000000,0.008050,0.003102,0.000502,0.006290,...,0.014644,0.034033,0.001468,0.0,0.002866,0.008035,0.010283,0.000000,0.071143,0.047574


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-13,0.007716,0.005819,NaN,0.008849,0.009505,0.013691,0.006692,0.016259,0.010941,0.0,...,0.006998,0.012958,0.016756,0.013085,0.0,0.011271,0.009542,0.011521,0.004991,0.004514
2021-01-14,0.008730,0.006710,NaN,0.011393,0.009507,0.014293,0.006552,0.004389,0.013279,0.0,...,0.007488,0.013665,0.008710,0.021883,0.0,0.012833,0.009157,0.007922,0.005627,0.008532
2021-01-15,0.007104,0.006040,NaN,0.014092,0.011810,0.011308,0.006179,0.008477,0.009726,0.0,...,0.008288,0.000314,0.008008,0.016818,0.0,0.011476,0.007728,0.013591,0.004822,0.004271


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-13      0.005634  0.008284  0.003574  0.009091  0.007012 -0.001034   
2021-01-14      0.015406  0.008917  0.006528  0.001897  0.009478  0.005176   
2021-01-15      0.005517  0.010163  0.009434  0.008045  0.008622  0.010299   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-13      0.001213  0.007218  0.013719  0.000638  ...  0.013829   
2021-01-14      0.010902  0.010982  0.008413  0.008923  ...  0.009378   
2021-01-15      0.005992  0.010955  0.007617  0.008212  ...  0.011824   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-13      0.008065  0.003319  0.006380   0.003738  0.002002  0.005669   
2021-01-14      0.018000  0.006983  0.001585   0.008690  0.030775  0.011274   
2021-01-15      0.011788  0.002190  0.001582   0.003692  0.010081  0.006689   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-13            NaN  0.006061  0.000000  
2021-01-14            NaN  0.008434  0.001664  
2021-01-15            NaN  0.003584  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-13,0.033555,0.004444,NaN,0.018222,0.015773,0.016914,0.008203,0.013490,0.003027,NaN,...,0.017101,0.012364,0.018987,0.000000,0.0,0.013694,0.012932,0.022644,0.008166,0.0
2021-01-14,0.032118,0.008850,NaN,0.017052,0.010033,0.020010,0.005866,0.002601,0.001006,NaN,...,0.010309,0.013301,0.007591,0.025316,0.0,0.013329,0.009901,0.018552,0.008628,0.0
2021-01-15,0.014298,0.004386,NaN,0.017043,0.008278,0.016492,0.005079,0.006257,0.007035,NaN,...,0.009597,0.000000,0.008219,0.006173,0.0,0.005332,0.006966,0.018214,0.007332,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-01-13         0.0  0.000000  0.000000  0.000000  0.012658     0.0   
2021-01-14         0.0  0.011429  0.028571 -0.020833  0.000000     0.0   
2021-01-15         0.0  0.011299  0.000000  0.000000  0.037500     0.0   

Province_State                                          ... Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...    Park Platte   
2021-01-13      0.000000  0.010256  0.000000  0.000000  ...     0.0    0.0   
2021-01-14      0.017544  0.000000  0.045455  0.034483  ...     0.0    0.0   
2021-01-15      0.000000  0.015228  0.000000  0.000000  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2021-01-13          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2021-01-14          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2021-01-15          0.0      0.0        0.0   0.0   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2021-01-13        0.0  
2021-01-14        0.0  
2021-01-15        0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-13,0.000557,0.009997,0.002308,0.011293,0.004321,0.003423,0.007001,0.002421,0.000474,0.004679,...,0.015492,0.028229,0.000649,0.0,0.003721,0.001371,0.005318,0.000781,0.042057,0.043118
2021-01-14,0.002061,0.010049,0.002452,0.008482,0.007262,0.024439,0.007280,0.002584,0.000394,0.006889,...,0.015322,0.040699,0.000717,0.0,0.004424,0.003973,0.005116,0.000628,0.048360,0.044481
2021-01-15,0.001551,0.009881,0.002458,0.008639,0.005404,0.012219,0.007122,0.002502,0.000546,0.005407,...,0.016207,0.036305,0.001175,0.0,0.004074,0.003619,0.004674,0.000551,0.046710,0.036846


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-13,0.002566,0.003863,0.001319,0.009127,0.004954,5.820766e-11,0.003358,0.003041,4.201208e-09,0.007569,...,0.015324,0.024973,0.000944,NaN,0.005638,3.652324e-42,0.013564,0.000307,0.030625,0.056017
2021-01-14,0.004108,0.003922,0.001192,0.010311,0.003656,1.000000e-01,0.003257,0.003554,2.100604e-09,0.007643,...,0.015041,0.021577,0.000472,NaN,0.005587,1.826162e-42,0.008893,0.000154,0.020224,0.067907
2021-01-15,0.004215,0.003547,0.001482,0.005155,0.005357,5.000000e-02,0.002759,0.003633,1.050302e-09,0.008590,...,0.014977,0.030432,0.001045,NaN,0.005087,9.130809e-43,0.011053,0.000077,0.022758,0.057538


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-13,0.006625,0.010946,0.002770,0.015752,0.029755,0.000548,0.005593,0.004343,0.000475,0.005847,...,0.017849,0.030757,0.000851,0.0,0.003729,0.003121,0.010010,0.001923,0.017353,0.021181
2021-01-14,0.008307,0.010440,0.002825,0.009177,0.026937,0.013432,0.007049,0.004165,0.000411,0.007426,...,0.019018,0.038337,0.000944,0.0,0.003580,0.001561,0.009709,0.000961,0.030376,0.049249
2021-01-15,0.005655,0.010048,0.002813,0.006134,0.016967,0.006716,0.007549,0.003633,0.000457,0.006858,...,0.016831,0.036185,0.001206,0.0,0.003223,0.004798,0.009996,0.000481,0.050759,0.048411


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-13,0.008104,0.004741,NaN,0.011935,0.010097,0.015096,0.005880,0.017403,0.010516,2.088976e-92,...,0.006850,0.012165,0.011881,0.015386,0.002990,0.011593,0.009581,0.011368,0.005056,0.007794
2021-01-14,0.008417,0.005726,NaN,0.011664,0.009802,0.014695,0.006216,0.010896,0.011898,1.044488e-92,...,0.007169,0.012915,0.010295,0.018634,0.001495,0.012213,0.009369,0.009645,0.005342,0.008163
2021-01-15,0.007760,0.005883,NaN,0.012878,0.010806,0.013002,0.006197,0.009686,0.010812,5.222440e-93,...,0.007729,0.006614,0.009152,0.017726,0.000747,0.011844,0.008548,0.011618,0.005082,0.006217


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-13      0.008303  0.008645  0.005608  0.009999  0.007294  0.003303   
2021-01-14      0.011855  0.008781  0.006068  0.005948  0.008386  0.004239   
2021-01-15      0.008686  0.009472  0.007751  0.006997  0.008504  0.007269   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-13      0.003635  0.008076  0.013036  0.005819  ...  0.013303   
2021-01-14      0.007269  0.009529  0.010724  0.007371  ...  0.011340   
2021-01-15      0.006630  0.010242  0.009171  0.007792  ...  0.011582   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-13      0.010423  0.004259  0.005476   0.005141  0.012009  0.009442   
2021-01-14      0.014211  0.005621  0.003531   0.006916  0.021392  0.010358   
2021-01-15      0.013000  0.003905  0.002556   0.005304  0.015737  0.008523   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-13           -1.0  0.007989  0.059929  
2021-01-14           -1.0  0.008212  0.030796  
2021-01-15           -1.0  0.005898  0.015398  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-13,0.029280,0.003646,NaN,0.019075,0.012686,0.017867,0.006410,0.010061,0.005345,NaN,...,0.015983,0.010186,0.015302,0.003646,0.000042,0.011640,0.013028,0.020100,0.007343,0.017414
2021-01-14,0.030699,0.006248,NaN,0.018064,0.011360,0.018939,0.006138,0.006331,0.003176,NaN,...,0.013146,0.011743,0.011447,0.014481,0.000021,0.012484,0.011465,0.019326,0.007986,0.008707
2021-01-15,0.022498,0.005317,NaN,0.017553,0.009819,0.017716,0.005608,0.006294,0.005105,NaN,...,0.011371,0.005872,0.009833,0.010327,0.000011,0.008908,0.009215,0.018770,0.007659,0.004353


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-13      0.005375  0.003310  0.000241  0.000173  0.015141  0.004267   
2021-01-14      0.002687  0.007369  0.014406 -0.010330  0.007570  0.002134   
2021-01-15      0.001344  0.009334  0.007203 -0.005165  0.022535  0.001067   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-13      0.008474  0.034620  0.002978  0.051990  ...  0.019729   
2021-01-14      0.013009  0.017310  0.024216  0.043236  ...  0.009865   
2021-01-15      0.006504  0.016269  0.012108  0.021618  ...  0.004932   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-13      0.000244  0.000004  0.000338   0.023260  0.000024  0.032366   
2021-01-14      0.000122  0.000002  0.000169   0.011630  0.000012  0.016183   
2021-01-15      0.000061  0.000001  0.000084   0.005815  0.000006  0.008092   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-13      -0.501961  0.024029  0.003581  
2021-01-14      -0.501961  0.012015  0.001790  
2021-01-15      -0.501961  0.006007  0.000895  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210115,AK,49835,NaN,1339391.0,NaN,totalTestsViral,1389226,77.0,1152.0,...,1389226,1,3,0d6d7a509fe581dba7c140188ff4c9016f79482c,0,0,0,0,0,NaN
1,20210115,AL,417528,85443.0,1680128.0,NaN,totalTestsPeopleViral,2012213,2863.0,38763.0,...,2097656,85,313,65f7f5ceabbef741fe718e564feec8468d699f4f,0,0,0,0,0,NaN
2,20210115,AR,267635,51296.0,2032273.0,NaN,totalTestsViral,2248612,1314.0,12598.0,...,2299908,35,95,1475b072612e89caa05010d39a0f687a77b9cf4b,0,0,0,0,0,NaN
3,20210115,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,69cae2d608316cfd548a55eff63c4061d569a705,0,0,0,0,0,NaN
4,20210115,AZ,658186,36409.0,2552128.0,NaN,totalTestsViral,5912271,4866.0,45665.0,...,3210314,185,405,87160dcbe7cfbaff9fa4e15320b291abde58afde,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210115,AK,49835.0,NaN,1339391.0,NaN,totalTestsViral,1389226.0,77.0,1152.0,...,1389226,1,3,0d6d7a509fe581dba7c140188ff4c9016f79482c,0,0,0,0,0,NaN
1,20210115,AL,417528.0,85443.0,1680128.0,NaN,totalTestsPeopleViral,2012213.0,2863.0,38763.0,...,2097656,85,313,65f7f5ceabbef741fe718e564feec8468d699f4f,0,0,0,0,0,NaN
2,20210115,AR,267635.0,51296.0,2032273.0,NaN,totalTestsViral,2248612.0,1314.0,12598.0,...,2299908,35,95,1475b072612e89caa05010d39a0f687a77b9cf4b,0,0,0,0,0,NaN
3,20210115,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,69cae2d608316cfd548a55eff63c4061d569a705,0,0,0,0,0,NaN
4,20210115,AZ,658186.0,36409.0,2552128.0,NaN,totalTestsViral,5912271.0,4866.0,45665.0,...,3210314,185,405,87160dcbe7cfbaff9fa4e15320b291abde58afde,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-15,AK,49835,NaN,1339391.0,NaN,totalTestsViral,1389226,77.0,1152.0,NaN,...,1389226,1,3,0d6d7a509fe581dba7c140188ff4c9016f79482c,0,0,0,0,0,NaN
2021-01-15,AL,417528,85443.0,1680128.0,NaN,totalTestsPeopleViral,2012213,2863.0,38763.0,NaN,...,2097656,85,313,65f7f5ceabbef741fe718e564feec8468d699f4f,0,0,0,0,0,NaN
2021-01-15,AR,267635,51296.0,2032273.0,NaN,totalTestsViral,2248612,1314.0,12598.0,434.0,...,2299908,35,95,1475b072612e89caa05010d39a0f687a77b9cf4b,0,0,0,0,0,NaN
2021-01-15,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,69cae2d608316cfd548a55eff63c4061d569a705,0,0,0,0,0,NaN
2021-01-15,AZ,658186,36409.0,2552128.0,NaN,totalTestsViral,5912271,4866.0,45665.0,1138.0,...,3210314,185,405,87160dcbe7cfbaff9fa4e15320b291abde58afde,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-15,AK,49835.0,NaN,1339391.0,NaN,totalTestsViral,1389226.0,77.0,1152.0,NaN,...,1389226,1,3,0d6d7a509fe581dba7c140188ff4c9016f79482c,0,0,0,0,0,NaN
2021-01-15,AL,417528.0,85443.0,1680128.0,NaN,totalTestsPeopleViral,2012213.0,2863.0,38763.0,NaN,...,2097656,85,313,65f7f5ceabbef741fe718e564feec8468d699f4f,0,0,0,0,0,NaN
2021-01-15,AR,267635.0,51296.0,2032273.0,NaN,totalTestsViral,2248612.0,1314.0,12598.0,434.0,...,2299908,35,95,1475b072612e89caa05010d39a0f687a77b9cf4b,0,0,0,0,0,NaN
2021-01-15,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,69cae2d608316cfd548a55eff63c4061d569a705,0,0,0,0,0,NaN
2021-01-15,AZ,658186.0,36409.0,2552128.0,NaN,totalTestsViral,5912271.0,4866.0,45665.0,1138.0,...,3210314,185,405,87160dcbe7cfbaff9fa4e15320b291abde58afde,0,0,0,0,0,NaN
